In [1]:
import keras
from context import * # imports the MDN layer 
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K
%matplotlib inline
import h5py
from keras.models import model_from_json
from keras.optimizers import Adam
from predictions import define_models, mdn_predict_sequence, predict_z
from midi_handling import switch_range, noteArrayToStream
from keras.utils import multi_gpu_model
from my_classes import MDNDataGenerator
from data_prep import one_hot_decode
from music21 import converter, instrument, note, chord, stream, midi, environment
# Only for GPU use:
#os.environ["CUDA_VISIBLE_DEVICES"]="1"


#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)
#K.set_session(sess)

mdn_dataset_path = "h5_files/mdn_dataset_b1.h5"
vae_weights_path = "weights/512_64/weights-improvement-2455-1.00.hdf5"

############################ MDN initalisation ###################################
SEQ_LEN = 16
BATCH_SIZE = 64
HIDDEN_UNITS = 512
EPOCHS = 400
SEED = 2345  # set random seed for reproducibility


OUTPUT_DIMENSION = 64
NUMBER_MIXTURES = 10

random.seed(SEED)
np.random.seed(SEED)
############################ VAE initalisation ###################################
n_features = 131
timesteps = 16
learning_rate = 0.0001
n_encoder_units = 512
n_decoder_units = n_encoder_units
latent_dim = 64
dropout=0.3
beta=1
############################ MUSESCORE intialisation ###############################
environment.set("musescoreDirectPNGPath", "/usr/bin/musescore")

K.clear_session()

Using TensorFlow backend.


### Decoder model

In [2]:
# Decoding Model
# Same as training model except for dimension and mixtures.

decoder = keras.Sequential()
decoder.add(keras.layers.LSTM(HIDDEN_UNITS, batch_input_shape=(1,1,OUTPUT_DIMENSION), return_sequences=True, stateful=True))
decoder.add(keras.layers.LSTM(HIDDEN_UNITS, stateful=True))
decoder.add(mdn.MDN(OUTPUT_DIMENSION, NUMBER_MIXTURES))
decoder.compile(loss=mdn.get_mixture_loss_func(OUTPUT_DIMENSION,NUMBER_MIXTURES), optimizer=keras.optimizers.Adam())
decoder.summary()


decoder.load_weights("weights/MDN_b1-weights-improvement-76-200.42.hdf5")
print("Loaded MDN weights from disk")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (1, 1, 512)               1181696   
_________________________________________________________________
lstm_2 (LSTM)                (1, 512)                  2099200   
_________________________________________________________________
mdn_1 (MDN)                  (1, 1290)                 661770    
Total params: 3,942,666
Trainable params: 3,942,666
Non-trainable params: 0
_________________________________________________________________
Loaded MDN weights from disk


## Get MusicVAE

In [3]:
# define model
train, infenc, infdec = define_models(n_encoder_units=n_encoder_units,
                                        n_decoder_units=n_decoder_units,
                                        latent_dim=latent_dim, 
                                      timesteps=timesteps,
                                      n_features=n_features,
                                      learning_rate=learning_rate,
                                        dropout=dropout,
                                      beta=beta,
                                        epsilon_std=1.)

train.load_weights(vae_weights_path)
print("Loaded VAE weights from disk")

all_weights = train.get_weights()
decoder_position = len(infenc.get_weights())
decoder_weights = all_weights[decoder_position::]
encoder_weights = all_weights[:decoder_position]

infenc.set_weights(encoder_weights)
infdec.set_weights(decoder_weights)

Loaded VAE weights from disk


## Predict

In [4]:
test_ID_list_dict = []
ID_list = np.load("ID_lists/test_ID_list.npy")

test_list = ID_list.item().get("test_labels")

input_songs = []
output_songs = []

ID = test_list[500]
encoder_input = np.load('encoder_inputs/' + ID + '.npy').reshape(1, SEQ_LEN, n_features)    

z = predict_z(infenc=infenc,source=encoder_input,latent_dim=OUTPUT_DIMENSION)

mdn_song = []

for i in range(25):

    params = decoder.predict(z.reshape(1,1,64))
    print(params.shape)
    z = mdn.sample_from_output(params[0], OUTPUT_DIMENSION, NUMBER_MIXTURES, temp=1.5)
    print(z.shape)
    song = mdn_predict_sequence(infdec=infdec,
                                 z=z,
                                 latent_dim=OUTPUT_DIMENSION,
                                 seq_len=SEQ_LEN,
                                 temperature=1,
                                 n_decoder_units=n_decoder_units)
    
    print(song.shape)
    mdn_song.append(one_hot_decode(song.reshape(SEQ_LEN, n_features)))

(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)
(16, 1, 131)
(1, 1290)
(1, 64)


KeyboardInterrupt: 

In [ ]:
mdn_song = np.ravel(mdn_song)
mdn_song = switch_range(mdn_song)

In [ ]:
melody_stream = noteArrayToStream(switch_range(list(np.ravel(mdn_song))))

melody_stream.show()

In [ ]:
len(mdn_song)